In [111]:
import numpy as np 
import matplotlib.pyplot as plt
import math
from scipy.fft import fft2, ifft2, rfft2, irfft2
import time
from time import strftime, gmtime
import numba
from numba import jit
import pandas as pd
import sys

import concurrent.futures
import multiprocessing
num_processes = multiprocessing.cpu_count()

from timeit import default_timer as timer

import sympy

font = {'family': 'serif',
          'color':  'black',
          'weight': 'normal',
          'size': 16,
          }

In [121]:
global first_term, second_term, third_term, V_ph, V_ph_k, rho, dt, U, dl

N = 512
k_max = N
L = 20

#del r_discretization, k_discretization
dl = L / N
l = np.linspace(-L,L,N)
X, Y = np.meshgrid(l,l)

k2 = np.zeros(N)
p = 2*np.pi/L
for i in range(0,N//2):
  k2[i] = pow(i*p,2) 
for i in range(N//2,N):
  k2[i] = pow((i-1)*p,2)

Kx2, Ky2 = np.meshgrid(k2,k2)

K2 = Kx2+Ky2
K = np.sqrt(K2)

g = np.ones((N,N), dtype=np.complex64)
S = np.ones((N,N), dtype=np.complex64)

#Initializing the terms
first_term = np.zeros((N,N), dtype=np.complex64)
second_term = np.zeros((N,N), dtype=np.complex64)
third_term = np.zeros((N,N), dtype=np.complex64)

V_ph = np.zeros((N,N), dtype=np.complex64) 
V_ph_k = np.zeros((N,N), dtype=np.complex64)

U = 0.0001
rho = 1

v = U*np.exp(-X*X-Y*Y)
c1 = pow(2*np.pi,-2) / rho

In [122]:
@numba.jit(nopython=True, parallel=False)
def primeira_derivada( dr, y_anterior, y_0):
  derivada = (y_0-y_anterior)/dr
  return derivada

@numba.jit(nopython=True, parallel=True)
def calculate_second_term(g):
  #Calculating the second gradient of |sqrt(g(r))|^2
  grad_x = np.zeros((N,N), dtype=np.complex64)
  grad_y = np.zeros((N,N), dtype=np.complex64)

  for i in range(0,N):
    for j in range(0,N-1):
        grad_x[i,j] = g[i,j] - g[i,j+1]
    grad_x[i,N-1] = grad_x[i,N-2] 

  for j in range(0,N):
    for i in range(0,N-1):
        grad_y[i,j] = g[i,j] - g[i+1,j]
    grad_y[N-1,j] = grad_y[N-2,j]  

  aux = (grad_x*grad_x+grad_y*grad_y) / (dl * dl)
  gradient_root_g = aux / (4.0*np.absolute(g))

  return gradient_root_g

In [124]:
g = np.ones((N,N), dtype=np.complex64)
S = np.ones((N,N), dtype=np.complex64)
aux = np.zeros((N,N))
dt = 0.001
condition = True
counter = 1
while condition:
  
  S = np.real(1 + rho * fft2(g-1))
  omega_k = -0.5*K2*(2*S+1)*(1-(1/S))*(1-(1/S))
  omega = ifft2(omega_k) * c1
  gradient_g = calculate_second_term(g)
  V_ph = g*v+2*gradient_g+(omega-1)*g
  V_ph_k = rho * fft2(V_ph)
  aux = np.real(K2+2*V_ph_k)
  #if any(np.less_equal(aux, 0)):
   # print("instability at rho = ", rho)
    #break
  new_S = K / np.sqrt(np.abs(aux))
  S = (1-dt)*S + dt*new_S
  new_g = np.real(1+fft2(S-1) * c1)
  error = np.sum(np.sum(g-new_g)) * dl * dl
  condition = error > 1e-6
  g = new_g
  print('i = {}, error = {}'.format(counter, error))
  counter = counter + 1

i = 1, error = (0.010131987917229506+0j)
i = 2, error = 67.21099696738129
i = 3, error = 445847.1665664084
i = 4, error = 2957547200.6726246
i = 5, error = 19619022167551.684
i = 6, error = 1.3014366456208942e+17
i = 7, error = 8.633138431161329e+20
i = 8, error = 5.726831146362648e+24
i = 9, error = nan


In [119]:
g = np.ones((N,N))
S = np.ones((N,N))
  
S = 1 + rho * rfft2(g-1)
S

array([[1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       ...,
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j]])

In [120]:
S.shape

(512, 257)